In [91]:
import pandas as pd

# Read the Excel file, skipping the first 3 rows
df = pd.read_excel("ввафав.xlsx", skiprows=3)

# Set the first row as the column names
df.columns = df.iloc[0]

# Drop the first row since it's now the column names
df = df[1:].reset_index(drop=True)

# Drop rows that contain only NaN values
df = df.dropna(how='all')

# Display the first 218 rows
df.head(3)

,"Регистрируемые (расчетные) параметры, единицы измерения",Name method,русское,латиницей,Baker Hughes,Halliburton,Schlumberger,Pathfinder,Sperry-Sun,Weatherford (Reeves),Синонимы
0,"Разность потенциалов электрического поля, мВ","Spontaneous Potential Logging, mV",ПС,PS,SP,SP,SP,-,-,"SP, SPR, SPCG, SPDL, SPLL, SPRL, CGSP","СП, PS, RSP"
1,"Разность потенциалов электрического поля, мВ","Spontaneous Gradient Potential Logging, mV",ПСГ,SPGL,-,-,-,-,-,-,-
2,"Разность потенциалов электрического поля, мВ","Electrode Potential Logging, mV",ЭП,EPL,-,-,-,-,-,-,-


In [92]:
df = df.fillna('-')

merged_text = df.iloc[:].apply(lambda row: ','.join(row[4::].astype(str)), axis=1)

# Add the merged text as a new column
df['Merged Text'] = pd.Series(merged_text).fillna('')

df = df.drop(df.columns[4:11], axis=1)



In [93]:
# Function to split and replace
def process_merged_text(text):
    if pd.notna(text):
        parts = text.split(',')
        return [part for part in parts if part != '-']
    return []

# Apply the function to the 'Merged Text' column
df['Merged Text'] = df['Merged Text'].apply(process_merged_text)
df

,"Регистрируемые (расчетные) параметры, единицы измерения",Name method,русское,латиницей,Merged Text
0,"Разность потенциалов электрического поля, мВ","Spontaneous Potential Logging, mV",ПС,PS,"[SP, SP, SP, SP, SPR, SPCG, SPDL, SPLL, S..."
1,"Разность потенциалов электрического поля, мВ","Spontaneous Gradient Potential Logging, mV",ПСГ,SPGL,[]
2,"Разность потенциалов электрического поля, мВ","Electrode Potential Logging, mV",ЭП,EPL,[]
3,"Разность потенциалов электрического поля, мВ","Two-electrode Potential Logging, mV",КПГП,TEPL,[]
4,"Разность потенциалов вызванной поляризации, мВ","Induced Potential Logging, mV",ВП,IPL,[]
...,...,...,...,...,...
706,Кривая восстановления давления,-,КВД,KVD,[]
707,Кривая стабилизации давления,-,КСД,KSD,[]
708,Кривая восстановления температуры,-,КВТ,KVT,[]
709,Кривая стабилизации температуры,-,КСТ,KST,[]


In [94]:
import re 

def split_and_clean(arr):
    if isinstance(arr, list):
        cleaned_parts = []
        for part in arr:
            match = re.match(r'(.*?)\s*\((.*?)\)', part)
            if match:
                cleaned_parts.append(match.group(1).strip())
                cleaned_parts.append(match.group(2).strip())
            else:
                cleaned_parts.append(part.strip())
        return cleaned_parts
    return []
df['Merged Text'] = df['Merged Text'].apply(split_and_clean)


In [95]:
df.columns = ["RU_COM", "ENG_COM", "RU", "ENG", "Merged"]

In [96]:
df

,RU_COM,ENG_COM,RU,ENG,Merged
0,"Разность потенциалов электрического поля, мВ","Spontaneous Potential Logging, mV",ПС,PS,"[SP, SP, SP, SP, SPR, SPCG, SPDL, SPLL, SPRL, ..."
1,"Разность потенциалов электрического поля, мВ","Spontaneous Gradient Potential Logging, mV",ПСГ,SPGL,[]
2,"Разность потенциалов электрического поля, мВ","Electrode Potential Logging, mV",ЭП,EPL,[]
3,"Разность потенциалов электрического поля, мВ","Two-electrode Potential Logging, mV",КПГП,TEPL,[]
4,"Разность потенциалов вызванной поляризации, мВ","Induced Potential Logging, mV",ВП,IPL,[]
...,...,...,...,...,...
706,Кривая восстановления давления,-,КВД,KVD,[]
707,Кривая стабилизации давления,-,КСД,KSD,[]
708,Кривая восстановления температуры,-,КВТ,KVT,[]
709,Кривая стабилизации температуры,-,КСТ,KST,[]


In [97]:
rus_com = {}
eng_com = {}

for index, row in df.iterrows():
    key = row["RU"]
    value = row["RU_COM"]    
    rus_com[key] = value
    
    key = row["ENG"]
    value = row["ENG_COM"]    
    eng_com[key] = value

In [98]:
rus = {}
eng = {}

for index, row in df.iterrows():
    value = row['RU']
    keys = [row['ENG']] + row['Merged'] + [row["RU"]]
    
    for k in keys:# Remove empty strings or empty lists
        rus[k] = value
        
for index, row in df.iterrows():
    value = row['ENG']
    keys = [row['ENG']] + row['Merged'] + [row["RU"]]
    
    for k in keys:# Remove empty strings or empty lists
        eng[k] = value
        

{'PS': 'PS', 'SP': 'PS', 'SPR': 'PS', 'SPCG': 'PS', 'SPDL': 'PS', 'SPLL': 'PS', 'SPRL': 'PS', 'CGSP': 'PS', 'СП': 'PS', 'RSP': 'PS', 'ПС': 'PS', 'SPGL': 'SPGL', 'ПСГ': 'SPGL', 'EPL': 'EPL', 'ЭП': 'EPL', 'TEPL': 'TEPL', 'КПГП': 'TEPL', 'IPL': 'IPL', 'ВП': 'IPL', 'SEL': 'SEL', 'КСК': 'SEL', 'REZ': 'REZ', 'DFRM': 'REZ', 'DPRM': 'REZ', 'AFMR': 'REZ', 'RMM5': 'REZ', 'MRES': 'MRES', 'MRES_SL': 'REZ', 'HICRS': 'REZ', 'ACRS': 'REZ', 'AHCRS': 'REZ', 'ASCRS': 'REZ', 'RMSF': 'VSM', 'Рс': 'REZ', 'РЕЗИСТ': 'REZ', 'Rc': 'REZ', 'RES': 'REZ', 'RM': 'REZ', 'RT': 'RT', 'RT_D': 'REZ', 'RB': 'REZ', 'Рез': 'MRES', 'BKZ': 'BKZ', 'БКЗ': 'BKZ', 'GZ1': 'GZ1', 'КС1': 'GZ1', 'A0.4M0.1N': 'GZ1', 'A04M01N': 'GZ1', 'ГЗ1': 'GZ1', 'GZ2': 'GZ2', 'КC2': 'GZ2', 'A1.0M0.1N': 'GZ2', 'A1M0.1N': 'GZ2', 'A1M01N': 'GZ2', 'ГЗ2': 'GZ2', 'GZ3': 'GZ3', 'КC3': 'GZ3', 'A2.0M0.5N': 'GZ3', 'A2M0.5N': 'GZ3', 'А2М05N': 'GZ3', 'ГЗ3': 'GZ3', 'GZ4': 'GZ4', 'КC4': 'GZ4', 'A4.0M0.5N': 'GZ4', 'A4M0.5N': 'GZ4', 'A4M05N': 'GZ4', 'ГЗ4': 'GZ4', 

In [100]:
import json



with open("eng_com.json", 'w') as json_file:
    json.dump(eng_com, json_file)
    
with open("rus_com.json", 'w') as json_file:
    json.dump(rus_com, json_file)
    
with open("eng.json", 'w') as json_file:
    json.dump(eng, json_file)
    
with open("rus.json", 'w') as json_file:
    json.dump(rus, json_file)
